In [1]:
import torch.nn as nn
import torch

- Sử dụng `CausalAttention` được implement ở phần trước.

In [2]:
class CausalAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.dim_out = dim_out
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(   # Đăng ký một tensor không phải là tham số của mô hình
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, dim_in = x.shape

        # 3 Q,K,V có shape (batch_size, context_length, dim_out)
        keys = self.W_key(x)    
        queries = self.W_query(x)
        values = self.W_value(x)

        # keys.transpose(1, 2) đổi shape từ (B, T, D) thành (B, D, T)
        attention_scores = queries @ keys.transpose(1, 2)  
        print(f"Batch Attention Scores:")
        print(attention_scores)
        # [:num_tokens, :num_tokens] -- Slicing mask để phù hợp với số token hiện tại
        attention_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf  # Dùng được từ register_buffer 
        )
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5, dim=-1
        )
        attention_weights = self.dropout(attention_weights)

        context_vectors = attention_weights @ values

        return context_vectors

- Wrapper class triển khai `multi-head attention`.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(dim_in, dim_out, context_length, dropout) for _ in range(num_heads)]
        )

    def forward(self, x):
        # dim=-1 để concat theo chiều cuối cùng, do mỗi head output shape (batch_size, num_tokens, dim_out)
        # dim_out là chiều của mỗi context vector
        return torch.cat([head(x) for head in self.heads], dim=-1)